## 크롤링에 필요한 라이브러리

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup as bs
import math
from pandas_profiling import ProfileReport

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 주피터 셀 넓이 조절 ---------------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', None)

# pip install selenium
# pip install openpyxl

## 크롬 웹브라우저 실행

In [2]:
driver = webdriver.Chrome("C:/AI/chromedriver_win32/chromedriver.exe")

## 네이버 로그인

In [3]:
# # 로그인 전용 화면
# driver.get('https://nid.naver.com/nidlogin.login')
# # 아이디와 비밀번호 입력
# driver.find_element_by_name('id').send_keys('mingiraffe')
# driver.find_element_by_name('pw').send_keys('rlfls9923!')
# # 로그인 버튼 클릭
# driver.find_element_by_css_selector('#log\.login').click()

## 사이트 주소

In [4]:
driver.get("https://cafe.naver.com/mbticafe")
time.sleep(2)

## 게시판 클릭  --> 게시판 루프 돌리기

In [5]:
keyword = 'INFP ♧ ENFP'
driver.find_element_by_link_text(keyword).click()

## 게시판 프레임 접근

In [6]:
driver.switch_to.frame("cafe_main")

## 게시글 50개씩

In [7]:
driver.find_element_by_css_selector("#listSizeSelectDiv").click()
driver.find_element_by_xpath("/html/body/div[1]/div/div[2]/div/div[3]/ul/li[7]").click()

In [8]:
#crawling_list = []
no_app = []
links_app  = []
crawling_no = int(input('클롤링 할 글 개수를 입력 :'))

클롤링 할 글 개수를 입력 : 5000


In [9]:
# 크롤링 해야 할 페이지 계산
crawling_page = int(math.ceil(crawling_no / 50)+1)   # 여기 수정

try: 
    for page in range(1,crawling_page):
        # 페이지 클릭
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
        
        no = [i.text for i in driver.find_elements_by_css_selector('.td_article')]
        no_split = [ni.split()[0] for ni in no]

        # href 링크 수집
        article_list = driver.find_elements_by_css_selector('.td_article > div.board-list > div > a.article')
        article_urls = [ i.get_attribute('href') for i in article_list ]    
            
        # 수집 데이터 append

        links_app.append(article_urls)
        
        # 10페이지 마다 프린트 & 다음 페이지로 클릭
        if str(page)[-1] == '0':
            print(int(page), 'page 크롤링 완료')
            driver.find_element_by_link_text('다음').click()
# 더이상 페이지가 존재하지 않을 시
except:
    print('더이상 페이지가 존재하지 않음')

# driver.close()
    
# 리스트안 리스트 분해
no_list = sum(no_app, [])

links_list = sum(links_app, [])

# 판다스화
df = pd.DataFrame({'번호':no_list,
                         '주소':links_list})
# 필독, 공지 삭제
df = df.drop(df[df['번호'] == '필독'].index)
df = df.drop(df[df['번호'] == '공지'].index)
df = df.reset_index(drop=True)

print('글 ', len(df), '개 크롤링 완료. \n크롤링 종료.', sep='')

df

# 저장
df.to_excel('crawler_naver cafe_게시판 {}.xlsx'.format(keyword))

10 page 크롤링 완료
20 page 크롤링 완료
30 page 크롤링 완료
40 page 크롤링 완료
50 page 크롤링 완료
60 page 크롤링 완료
70 page 크롤링 완료
80 page 크롤링 완료
90 page 크롤링 완료
100 page 크롤링 완료


ValueError: arrays must all be same length

In [ ]:
df

In [ ]:
# 저장
df.to_excel('crawler_naver cafe_게시판 {}.xlsx'.format(keyword))

----

In [ ]:
# 요기까지
asdfsdfasdfasdfsdfsdfsdaf

----

In [ ]:
print(df['주소'])

In [ ]:
add = df['주소']
add = add.values.tolist()

In [ ]:
add

## 클릭된 게시글로 접근

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import urllib.parse
from urllib.request import urlopen

In [ ]:
res_list = []
# Beautifulsoup 활용
for link in add:
    driver.get(link)

    driver.switch_to.frame("cafe_main")
    html = driver.page_source
    soup = bs(html, "html.parser")
    


In [ ]:
    # 게시글에서 제목 추출
title = soup.select('#app > div > div > div.ArticleContentBox > div.article_header > div.ArticleTitle > div > h3')
    # 내용을 하나의 텍스트로 만든다. (띄어쓰기 단위)
content_tags = soup.select('#app > div > div > div.ArticleContentBox > div.article_container > div.article_viewer')
content = ' '.join([ tags.get_text() for tags in content_tags ])
# dict형태로 만들어 결과 list에 저장
res_list.append({'title' : title, 'content' : content})
# time.sleep 작업도 필요하다.
# 결과 데이터프레임화
cafe_df = pd.DataFrame(res_list)
# csv파일로 추출
cafe_df.to_csv('cafe_crawling.csv', index=False)

In [ ]:
driver.get(link)

In [ ]:
# href 속성을 찾아 url을 리스트로 저장한다.
    ## 각 카페마다 태그의 형식이 다를 수 있으니 확인해야 한다. ( 태그.클래스명 )
    article_list = driver.find_elements_by_css_selector('span.aaa > a.m-tcol-c')
    article_urls = [ i.get_attribute('href') for i in article_list ]

## 카페 게시글 내용 수집

In [ ]:
import pickle
from contextlib import suppress

for keyword in cafe.get("keywords"):
    df = pickle.load(open(save_path+"cafe_address_"+cafe.get("name")+"_"+keyword+".pkl", 'rb'))

    i=0
    contents_list = []   # 내용
    reply_list = []      # 댓글
    error_list = []      # 에러난 게시글

    while True:

        ### 수집 링크로 이동
        url = "https://cafe.naver.com"+df.loc[i,'link']
        idx_no = df.loc[i,'idx_no']    # 인덱스 번호
        driver.get(url)
        time.sleep( random.randint(2,5) )
        try:
            driver.switch_to.frame('cafe_main')
            time.sleep( random.randint(2,5) )
            r = driver.page_source
            page_soup = BeautifulSoup(r, "html.parser")
            content = page_soup.find('div', class_='ArticleContentBox')  

            ### 게시글 수집
            temp_dict={}
            temp_dict['idx_no'] = idx_no
            temp_dict['title'] = ""
            with suppress(AttributeError):   # 제목 없는 게시글
                temp_dict['title'] = content.find('h3',class_='title_text').text.strip()
            temp_dict['content'] = content.find("div", class_="article_viewer").text.strip()
            temp_dict['nick'] = content.find('div',class_='profile_info').find('a',class_='nickname').text.strip()
            temp_dict['date'] = content.find('div',class_='article_info').find('span',class_='date').text.strip()
            temp_dict['view'] = ""
            with suppress(AttributeError):
                temp_dict['view'] = content.find('div',class_='article_info').find('span',class_='count').text.strip()
            contents_list.append(temp_dict)

            ### 댓글 수집
            if content.find("div", class_="ReplyBox") is not None:   # 댓글 기능이 아예 없음  
                comment_num = content.find("div", class_="ReplyBox").find("a",class_="button_comment").find("strong").text
                if comment_num!='0':   # 댓글이 없음
                    comment = content.find("div", class_="CommentBox").find("ul",class_="comment_list").select("li")
                    
                    ### 댓글 구분
                    com_n=0    # 댓글
                    com_nn=0   # 대댓글
                    
                    for n in range(len(comment)):

                        if comment[n].get('class')==['CommentItem']:    # 댓글
                            com_n+=1; com_nn=0;
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn=1
                        elif comment[n].get('class')==['CommentItem', 'CommentItem--reply']:    # 대댓글
                            com_thread = str(com_n)+"-"+str(com_nn)
                            com_nn+=1

                        ### 댓글 내용 수집    
                        if comment[n].text.strip() != '삭제된 댓글입니다.':
                            com_nick = comment[n].find("a",class_="comment_nickname").text.strip()
                            com_date = comment[n].find("span",class_="comment_info_date").text.strip()
                            com_reply = comment[n].find("div",class_="comment_text_box").text.strip()
                            reply_list.append({'idx_no':idx_no, 'nick':com_nick, 'date':com_date, 'reply':com_reply, "thread":com_thread})
            i+=1

        except:
            i+=1
            ### 게시글을 볼 등급이 안됨
            if page_soup.find('strong', class_='emph') is not None:
                error_list.append({"error" :  page_soup.find('strong', class_='emph').text+"등급 필요"
                                   , "url" : url})
                pass
            ### 에러 따로 확인
            else:
                error_list.append({"error" : "에러 확인 필요"
                                   , "url" : url})
                pass

        ### 수집한 글 갯수만큼 반복
        if i == len(df):
            contents_df = pd.DataFrame(contents_list)
            contents_df.to_pickle("../../data/cafe/cafe_contents_"+cafe.get("name")+"_"+keyword+".pkl")
            reply_df = pd.DataFrame(reply_list)
            reply_df.to_pickle("../../data/cafe/cafe_replies_"+cafe.get("name")+"_"+keyword+".pkl")
            print("(현재시각) "+str(datetime.datetime.now())+": done")
            break
    
# 크롬 종료 
driver.quit()